# Expert F# 4.0 - Chapter 16 - Language oriented programming
  
**Expert F# 4.0**   
Authors: Syme, Don, Granicz, Adam, Cisternino, Antonio   
http://www.apress.com/us/book/9781484207413  
https://github.com/Apress/expert-fsharp-4.0/blob/master/ExpertFSharp-master/16Language/Script.fsx  
Nada Amin:
https://github.com/namin/spots/blob/master/probabilisticModeling/probabilisticModeling.fsx  
**Stochastic Lambda Calculus and Monads of Probability Distribution**      
Norman Ramsey and Avi Pfeffer  
http://www.cs.tufts.edu/~nr/pubs/pmonad-abstract.html  
**Practical Probabilistic Programming with Monads**   
Adam Scibior, 
Zoubin Ghahramani, 
Andrew D. Gordon   
http://mlg.eng.cam.ac.uk/pub/pdf/SciGhaGor15.pdf   
** Probabilistic Programming: What It Is and How It Works** (in Scala)  
Noel Welsh   
https://www.youtube.com/watch?v=e1Ykk_CqKTY  
A Probabilistic Functional Programming Language DSL in Scala:   
https://github.com/noelwelsh/pfennig     

See also:    
https://www.google.ca/search?q=probability+monad   
https://people.cs.kuleuven.be/~tom.schrijvers/Research/talks/probability_monad.pdf  
http://jliszka.github.io/2013/08/12/a-frequentist-approach-to-probability.html  
https://www.google.ca/search?q=free+monad    
- Avi Pfeffer - Practical Probabilistic Programming with Figaro - MLconf SEA 2016   
  - https://www.youtube.com/watch?v=eO4ZXLQjba8  
  - http://www.slideshare.net/SessionsEvents/avi-pfeffer-principal-scientist-charles-river-analytics  
- CUFP 2013: Avi Pfeffer: Functional Probabilistic Programming  
  - https://www.youtube.com/watch?v=U67guma2H6s  
  - http://cufp.org/2013/slides/pfeffer.pdf  

# Built-in workflows

## Sequences

In [1]:
seq {for i in 0 .. 3 -> (i, i * i)}
//val it : seq<int * int> = seq [(0, 0); (1, 1); (2, 4); (3, 9)]

seq [(0, 0); (1, 1); (2, 4); (3, 9)]

## Asynchronous processes

In [2]:
type Agent<'T> = MailboxProcessor<'T>

let counter =
    new Agent<_>(fun inbox ->
        let rec loop n =
            async {printfn "n = %d, waiting..." n
                   let! msg = inbox.Receive()
                   return! loop (n + msg)}
        loop 0)

# Workflow builder

## Helper functions

In [ ]:
type Attempt<'T> = (unit -> 'T option)

In [8]:
let succeed x = (fun () -> Some(x))
let fail = (fun () -> None)
let runAttempt (a : Attempt<'T>) = a()
let bind p rest = match runAttempt p with None -> fail | Some r -> (rest r)
let delay f = (fun () -> runAttempt (f()))
let combine p1 p2 = (fun () -> match p1() with None -> p2() | res -> res)

## Builder class

In [4]:
type AttemptBuilder() =
    /// Used to de-sugar uses of 'let!' inside computation expressions.
    member b.Bind(p, rest) = bind p rest
    /// Delays the construction of an attempt until just before it is executed
    member b.Delay(f) = delay f
    /// Used to de-sugar uses of 'return' inside computation expressions.
    member b.Return(x) = succeed x
    /// Used to de-sugar uses of 'return!' inside computation expressions.
    member b.ReturnFrom(x : Attempt<'T>) = x
    /// Used to de-sugar uses of 'c1; c2' inside computation expressions.
    member b.Combine(p1 : Attempt<'T>, p2 : Attempt<'T>) = combine p1 p2
    /// Used to de-sugar uses of 'if .. then ..' inside computation expressions when
    /// the 'else' branch is empty
    member b.Zero() = fail

Create an instance of the builder class, so we can create computation expressions `attempt{}`

In [5]:
let attempt = new AttemptBuilder()

### Examples: `alwaysOne`, `alwaysPair`, `failIfBig`, `failIfEitherBig`

In [6]:
let alwaysOne = attempt { return 1 }
let alwaysPair = attempt { return (1,"two") }
let failIfBig n = attempt {if n > 1000 then return! fail else return n}
let failIfEitherBig (inp1, inp2) = 
    attempt {
        let! n1 = failIfBig inp1
        let! n2 = failIfBig inp2
        return (n1, n2)}

#### Test

In [9]:
runAttempt alwaysOne
//val it : int option = Some 1

Some 1

In [10]:
runAttempt alwaysPair
//val it : (int * string) option = Some (1, "two")

Some (1, "two")

In [11]:
runAttempt (failIfBig 999)
//val it : int option = Some 999

Some 999

In [12]:
runAttempt (failIfBig 1001)
//val it : int option = None

None

In [13]:
runAttempt (failIfEitherBig (999, 998))
//val it : (int * int) option = Some(999,998)

Some (999, 998)

In [14]:
runAttempt (failIfEitherBig (1003, 998))
//val it : (int * int) option = None

None

In [8]:
runAttempt (failIfEitherBig (999, 1001))
//val it : (int * int) option = None

None

### `sumIfBothSmall` & variants

In [19]:
let sumIfBothSmall (inp1, inp2) = 
    attempt { 
        let! n1 = failIfBig inp1
        printfn "Hey, n1 was small!"
        let! n2 = failIfBig inp2
        printfn "n2 was also small!"
        let sum = n1 + n2
        return sum 
    }

In [20]:
runAttempt(sumIfBothSmall (999, 999))
//Hey, n1 was small!
//n2 was also small!
//val it : int option = Some 1998

Hey, n1 was small!
n2 was also small!


Some 1998

In [21]:
runAttempt(sumIfBothSmall (999, 1001))
//Hey, n1 was small!
//val it : int option = None

Hey, n1 was small!


None

In [22]:
let sumIfBothSmall' (inp1, inp2) = 
    attempt { let sum = ref 0
              let! n1 = failIfBig inp1
              sum := sum.Value + n1
              let! n2 = failIfBig inp2
              sum := sum.Value + n2
              return sum.Value}
//val sumIfBothSmall' : inp1:int * inp2:int -> (unit -> int option)

In [23]:
let sumIfBothSmall'' (inp1, inp2) = 
    attempt { let sum = ref 0
              let! n1 = failIfBig inp1
              sum := !sum + n1
              let! n2 = failIfBig inp2
              sum := !sum + n2
              return sum.Value}
//val sumIfBothSmall'' : inp1:int * inp2:int -> (unit -> int option)

In [24]:
runAttempt(sumIfBothSmall' (999, 999))
//val it : int option = Some 1998

Some 1998

In [25]:
runAttempt(sumIfBothSmall' (999, 1001))
//val it : int option = None

None

### `printThenSeven` & variants

In [26]:
let printThenSeven = attempt {printf "starting..."; return 3 + 4}
let printThenSeven' = attempt.Delay(fun () ->
    printf "starting..."; attempt.Return(3 + 4))
let printThenSeven'' = attempt.Delay(fun () ->
    printf "starting..."
    attempt.Return(3 + 4))

In [27]:
runAttempt printThenSeven
//starting...val it : int option = Some 7

starting...

Some 7

In [28]:
runAttempt printThenSeven'
//starting...val it : int option = Some 7

starting...

Some 7

In [29]:
runAttempt printThenSeven''
//starting...val it : int option = Some 7

starting...

Some 7

### `condition`

In [10]:
let condition p guard = (fun () -> 
    match p() with 
    | Some x when guard x -> Some x 
    | _ -> None)

In [11]:
type AttemptBuilder with
    [<CustomOperation("condition",MaintainsVariableSpaceUsingBind = true)>]
    member x.Condition(p, [<ProjectionParameter>] b) = condition p b

In [12]:
let attempt2 = new AttemptBuilder()

# Distributions

In [13]:
let random = new System.Random()
let rand() = random.NextDouble()

let randomNumberInCircle = attempt {
    let x, y = rand(), rand()
    condition (x * x + y * y < 1.0)
    return (x, y)}

In [14]:
type Distribution<'T when 'T : comparison> =
    abstract Sample : 'T
    abstract Support : Set<'T>
    abstract Expectation: ('T -> float) -> float

### Uniform

In [15]:
let always x =
    { new Distribution<'T> with
          member d.Sample = x
          member d.Support = Set.singleton x
          member d.Expectation(H) = H(x) }

### Bernoulli

In [ ]:
let rnd = System.Random()

let coinFlip (p : float) (d1 : Distribution<'T>) (d2 : Distribution<'T>) =
    if p < 0.0 || p > 1.0 then failwith "invalid probability in coinFlip"
    { new Distribution<'T> with
          member d.Sample =
              if rnd.NextDouble() < p then d1.Sample else d2.Sample
          member d.Support = Set.union d1.Support d2.Support
          member d.Expectation(H) =
             p * d1.Expectation(H) + (1.0 - p) * d2.Expectation(H) }

## Workflow

In [ ]:
let bindD (dist : Distribution<'T>) (k : 'T -> Distribution<'U>) =
    { new Distribution<'U> with
         member d.Sample = 
             (k dist.Sample).Sample
         member d.Support =
             Set.unionMany (dist.Support |> Set.map (fun d -> (k d).Support))
         member d.Expectation H = 
             dist.Expectation(fun x -> (k x).Expectation H) }

In [ ]:
type DistributionBuilder() =
    member x.Delay f = bindD (always ()) f
    member x.Bind(d, f) = bindD d f
    member x.Return v = always v
    member x.ReturnFrom vs = vs

In [ ]:
let dist = new DistributionBuilder()
let weightedCases (inp : ('T * float) list) =
    let rec coinFlips w l =
        match l with
        | [] -> failwith "no coinFlips"
        | [(d, _)] -> always d
        | (d, p) :: rest -> coinFlip (p / (1.0 - w)) (always d) (coinFlips (w + p) rest)
    coinFlips 0.0 inp

let countedCases inp =
    let total = Seq.sumBy (fun (_, v) -> v) inp
    weightedCases (inp |> List.map (fun (x, v) -> (x, float v / float total)))

### Roulette

In [ ]:
type Outcome = Even | Odd | Zero
let roulette = countedCases [ Even,18; Odd,18; Zero,1]

In [ ]:
roulette.Sample
//val it : Outcome = Even

roulette.Sample
//val it : Outcome = Odd

roulette.Expectation (function Even -> 10.0 | Odd -> 0.0 | Zero -> 0.0)
//val it : float = 4.864864865

### Traffic lights

In [ ]:
type Light = Red | Green | Yellow
let trafficLightD = weightedCases [Red, 0.50; Yellow, 0.10; Green, 0.40]
type Action = Stop | Drive

let cautiousDriver light = 
    dist { match light with
           | Red -> return Stop
           | Yellow -> return! weightedCases [Stop, 0.9; Drive, 0.1]
           | Green -> return Drive}

let aggressiveDriver light = 
    dist { match light with
           | Red -> return! weightedCases [Stop, 0.9; Drive, 0.1]
           | Yellow -> return! weightedCases [Stop, 0.1; Drive, 0.9]
           | Green -> return Drive} 

let otherLight light =
    match light with
    | Red -> Green
    | Yellow -> Red
    | Green -> Red 

type CrashResult = Crash | NoCrash

In [ ]:
// Where the suffix D means distribution
let crash (driverOneD, driverTwoD, lightD) = 
    dist { // Sample from the traffic light
           let! light = lightD

           // Sample the first driver's behavior given the traffic light
           let! driverOne = driverOneD light

           // Sample the second driver's behavior given the traffic light
           let! driverTwo = driverTwoD (otherLight light)

           // Work out the probability of a crash
           match driverOne, driverTwo with
           | Drive, Drive -> return! weightedCases [Crash, 0.9; NoCrash, 0.1]
           | _ -> return NoCrash}

In [ ]:
let model = crash (cautiousDriver, aggressiveDriver, trafficLightD)

In [ ]:
model.Sample
//val it : CrashResult = NoCrash

model.Sample
//val it : CrashResult = Crash

model.Expectation (function Crash -> 1.0 | NoCrash -> 0.0)
//val it : float = 0.0369

### `linesOfFile`

In [ ]:
let linesOfFile fileName = 
    seq { use textReader = System.IO.File.OpenText(fileName)
          while not textReader.EndOfStream do
              yield textReader.ReadLine()} 

### Random walk

In [ ]:
let rnd = System.Random()

let rec randomWalk k = 
    seq { yield k
          yield! randomWalk (k + rnd.NextDouble() - 0.5) }


randomWalk 10.0
//val it : seq<float> = seq [10.0; 10.44456912; 10.52486359; 10.07400056; ...]

randomWalk 10.0
//val it : seq<float> = seq [10.0; 10.03566833; 10.12441613; 9.922847582; ...]